In [11]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy import stats
from networkx.algorithms.community import greedy_modularity_communities
from community import community_louvain

In [12]:
def criar_rede_por_txt(arquivo_caminho, ponderado = False):
  G = nx.Graph()
  with open(arquivo_caminho,"r") as arquivo:
    for linha in arquivo: 
      linha = linha.replace("\n", "").split(" ")
      if ponderado:
        G.add_edge(linha[0], linha[1], peso = linha[2])
      else:
        G.add_edge(linha[0], linha[1])
  return G

def preprocessar_grafo(G, para_não_direcionado = True, remover_selfloop = True, componente_maior = True, renomear = True):
  if para_não_direcionado:
    G = G.to_undirected()
  if remover_selfloop:
    G.remove_edges_from(nx.selfloop_edges(G))
  if componente_maior:
    Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
    G = G.subgraph(Gcc[0])
  if renomear:
    G = nx.convert_node_labels_to_integers(G, first_label=0)
  return G

def calcular_modularidade(G, c):
  A = nx.adjacency_matrix(G)
  N = len(G)
  M = G.number_of_edges()
  Q = 0
  for i in np.arange(0,N):
      ki = len(list(G.neighbors(i)))
      for j in np.arange(0,N):
          if(c[i]==c[j]):
              kj = len(list(G.neighbors(j)))
              Q = Q + A[i,j]-(ki*kj)/(2*M)
  Q = Q/(2*M)
  return Q

def agrupamento_greedy(G):

  c = list(greedy_modularity_communities(G))
  comunidades = np.zeros(len(G.nodes()))
  nc = 0
  for k in range(0,len(c)):
    lista = list(map(int, c[k]))
    comunidades[sorted(lista)]=nc
    nc = nc+1
  
  return calcular_modularidade(G,comunidades)

def louvain(G):

  particao = community_louvain.best_partition(G)

  comunidades = np.array(list(particao.values()))
    
  return calcular_modularidade(G,comunidades)


Q1

In [13]:
caminho_g1 = r"data/jazz.txt"
G1 = criar_rede_por_txt(caminho_g1)
G1 = preprocessar_grafo(G1)


In [14]:
mod_g1 = agrupamento_greedy(G1)
print(mod_g1)


0.4389078153753761


Q2

In [15]:
mod_g2 = louvain(G1)
print(mod_g2)

0.43524242129214225


Q3

In [16]:
N = 128
tau1 = 3
tau2 = 1.5
k = 16
minc = 32
maxc = 32
G1 = nx.LFR_benchmark_graph(n = N, tau1 = tau1, tau2 = tau2, mu = 0.05, min_degree = k, max_degree = k, min_community=minc, max_community = maxc, seed = 10)
print("Q(mu = 0.05) =",  louvain(G1))
G2 = nx.LFR_benchmark_graph(n = N, tau1 = tau1, tau2 = tau2, mu = 0.1, min_degree = k, max_degree = k, min_community=minc, max_community = maxc, seed = 10)
print("Q(mu = 0.1) =",  louvain(G2))
G3 = nx.LFR_benchmark_graph(n = N, tau1 = tau1, tau2 = tau2, mu = 0.2, min_degree = k, max_degree = k, min_community=minc, max_community = maxc, seed = 10)
print("Q(mu = 0.2) =",  louvain(G3))

Q(mu = 0.05) = 0.643639535397838
Q(mu = 0.1) = 0.5427818606053953
Q(mu = 0.2) = 0.44487909975820294


Q4

In [17]:
G1 = nx.LFR_benchmark_graph(n = N, tau1 = tau1, tau2 = tau2, mu = 0.05, min_degree = k, max_degree = k, min_community=minc, max_community = maxc, seed = 10)
print("Q(mu = 0.05) =",  louvain(G1))
G2 = nx.LFR_benchmark_graph(n = N, tau1 = tau1, tau2 = tau2, mu = 0.2, min_degree = k, max_degree = k, min_community=minc, max_community = maxc, seed = 10)
print("Q(mu = 0.2) =",  louvain(G2))
G3 = nx.LFR_benchmark_graph(n = N, tau1 = tau1, tau2 = tau2, mu = 0.4, min_degree = k, max_degree = k, min_community=minc, max_community = maxc, seed = 10)
print("Q(mu = 0.4) =",  louvain(G3))

Q(mu = 0.05) = 0.643639535397838
Q(mu = 0.2) = 0.44487909975820294
Q(mu = 0.4) = 0.18248436216158032
